## สรุปการเปลี่ยน Embedding Model

### ✅ ผลลัพธ์ที่ได้:
1. **คอลเลกชันใหม่**: `combined_collection_thai` ถูกสร้างขึ้นสำเร็จ
2. **Embedding Model**: เปลี่ยนจาก `all-MiniLM-L6-v2` เป็น `sentence-transformers/distiluse-base-multilingual-cased`
3. **จำนวนเอกสาร**: 300 เอกสาร (ย้ายมาจากคอลเลกชันเดิม)
4. **Embedding Dimensions**: เพิ่มจาก 384 เป็น 512 dimensions

### 📈 ข้อดีของ Embedding ใหม่:
- **รองรับภาษาไทยดีกว่า**: ออกแบบมาสำหรับหลายภาษารวมถึงภาษาไทย  
- **ความแม่นยำสูงขึ้น**: จับความหมายของคำและประโยคภาษาไทยได้ดีกว่า
- **การค้นหาที่ดีขึ้น**: ผลการค้นหาจะตรงกับความต้องการมากขึ้น

### 🚀 การใช้งานต่อไป:
ใช้ `new_combined_vector_store` สำหรับการค้นหาและ RAG ในระบบ Chatbot เพื่อให้ได้ผลลัพธ์ที่เหมาะสมกับภาษาไทยมากขึ้น

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

generator_llm = ChatGroq(
    model="openai/gpt-oss-20b",  # Fastest, cheapest model
    temperature=0.1,
    max_tokens=2048,  # LIMIT: 2048 tokens per request
    timeout=30,      # Short timeout
    max_retries=1,   # Fewer retries
)

# Use HuggingFace embeddings that produce 384 dimensions
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased"  # This produces 384-dimensional embeddings
)

In [ ]:
# ฟังก์ชันสำหรับแปลงทุก collections และรวมเข้าด้วยกัน
def migrate_all_collections_to_new_embedding(target_collection_name, new_embedding_function):
    """
    แปลงทุก collections ไปยัง embedding ใหม่และรวมเข้าในคอลเลกชันเดียว
    """
    print("=== เริ่มการแปลงและรวม Collections ===")

    # ลบคอลเลกชันเป้าหมายถ้ามีอยู่แล้ว
    try:
        client.delete_collection(target_collection_name)
        print(f"ลบคอลเลกชัน {target_collection_name} เก่าแล้ว")
    except:
        print(f"ไม่พบคอลเลกชัน {target_collection_name} เก่า")

    # สร้างคอลเลกชันใหม่
    combined_vector_store = Chroma(
        client=client,
        collection_name=target_collection_name,
        embedding_function=new_embedding_function,
    )

    # รายการคอลเลกชันที่ต้องการแปลง (ยกเว้นคอลเลกชันเป้าหมาย)
    source_collections = ["attraction", "hotel", "restaurant"]

    total_documents = 0
    doc_counter = 0

    for collection_name in source_collections:
        try:
            print(f"\\n--- กำลังประมวลผล {collection_name} ---")

            # ดึงคอลเลกชันต้นฉบับ
            source_collection = client.get_collection(collection_name)
            collection_data = source_collection.get(include=['documents', 'metadatas'])

            documents = collection_data['documents']
            metadatas = collection_data['metadatas']
            document_count = len(documents)

            print(f"พบเอกสาร {document_count} รายการใน {collection_name}")

            if document_count == 0:
                print(f"ข้าม {collection_name} (ไม่มีเอกสาร)")
                continue

            # แปลงข้อมูลทีละ batch
            batch_size = 50
            total_batches = document_count // batch_size + (1 if document_count % batch_size > 0 else 0)

            for i in range(0, document_count, batch_size):
                batch_num = i // batch_size + 1
                print(f"  Batch {batch_num}/{total_batches} ของ {collection_name}")

                end_idx = min(i + batch_size, document_count)
                batch_documents = documents[i:end_idx]
                batch_metadatas = metadatas[i:end_idx] if metadatas else None

                # สร้าง Document objects พร้อมเพิ่ม source type
                doc_objects = []
                batch_ids = []

                for j, doc_text in enumerate(batch_documents):
                    metadata = batch_metadatas[j] if batch_metadatas else {}
                    # เพิ่มข้อมูล source collection
                    metadata['source_collection'] = collection_name

                    doc_objects.append(Document(page_content=doc_text, metadata=metadata))
                    batch_ids.append(f"{collection_name}_{doc_counter}")
                    doc_counter += 1

                # เพิ่มเอกสารลงในคอลเลกชันรวม
                try:
                    combined_vector_store.add_documents(documents=doc_objects, ids=batch_ids)
                except Exception as e:
                    print(f"    ข้อผิดพลาด batch {batch_num}: {e}")
                    continue

            total_documents += document_count
            print(f"เพิ่ม {document_count} เอกสารจาก {collection_name} เรียบร้อย")

        except Exception as e:
            print(f"ข้อผิดพลาดกับ {collection_name}: {e}")
            continue

    # ตรวจสอบผลลัพธ์
    final_count = client.get_collection(target_collection_name).count()
    print(f"\\n=== สรุปผลลัพธ์ ===")
    print(f"รวมเอกสารทั้งหมด: {total_documents} รายการ")
    print(f"เอกสารในคอลเลกชันใหม่: {final_count} รายการ")
    print(f"คอลเลกชัน '{target_collection_name}' สร้างเสร็จสิ้น")

    return combined_vector_store

# เรียกใช้ฟังก์ชันเพื่อสร้างคอลเลกชันรวมใหม่
print("เริ่มการแปลง Collections ทั้งหมดไปยัง Multilingual Embedding...")
all_combined_vector_store = migrate_all_collections_to_new_embedding(
    target_collection_name="all_collections_thai",
    new_embedding_function=new_embeddings
)

In [ ]:
# ทดสอบการค้นหาในคอลเลกชันรวมใหม่
test_queries_comprehensive = [
    "แนะนำสถานที่ท่องเที่ยวธรรมชาติในเชียงใหม่",
    "โรงแรมดีๆ ในเชียงใหม่",
    "ร้านอาหารเหนือแท้ๆ",
    "ตลาดน่าเที่ยว",
    "วัดสวยๆ ในเชียงใหม่"
]

print("=== ทดสอบการค้นหาในคอลเลกชันรวม (all_collections_thai) ===")
print(f"Embedding: {new_embeddings.model_name}")
print("="*70)

for i, query in enumerate(test_queries_comprehensive, 1):
    print(f"\\n{i}. คำถาม: {query}")
    print("-" * 50)

    # ค้นหาผลลัพธ์
    results = all_combined_vector_store.similarity_search(query, k=5)

    # แสดงผลจำแนกตาม source collection
    source_counts = {}
    for j, doc in enumerate(results, 1):
        name = doc.metadata.get('name', 'ไม่มีชื่อ')
        source = doc.metadata.get('source_collection', 'ไม่ระบุ')
        rating = doc.metadata.get('rating', 'N/A')

        # นับจำนวนตาม source
        source_counts[source] = source_counts.get(source, 0) + 1

        print(f"  {j}. [{source.upper()}] {name}")
        if rating != 'N/A':
            print(f"      คะแนน: {rating}")
        print(f"      {doc.page_content[:100]}...")

    # แสดงสถิติ source
    print(f"\\n  📊 แหล่งข้อมูล: {dict(source_counts)}")

print(f"\\n🎉 การทดสوบเสร็จสิ้น! คอลเลกชัน 'all_collections_thai' พร้อมใช้งาน")

In [ ]:
# ตรวจสอบสถานะคอลเลกชันสุดท้าย
print("=== สถานะคอลเลกชันทั้งหมดหลังการแปลง ===")
print()

collections = client.list_collections()
for collection in collections:
    try:
        col = client.get_collection(collection.name)
        count = col.count()

        # ตรวจสอบว่าเป็นคอลเลกชันใหม่หรือไม่
        is_new = "_thai" in collection.name or collection.name == "all_collections_thai"
        status = "🆕 ใหม่ (Multilingual)" if is_new else "📄 เดิม"

        print(f"{status} {collection.name}: {count:,} documents")

        # แสดงตัวอย่าง metadata ของคอลเลกชันใหม่
        if is_new and count > 0:
            sample = col.get(limit=1, include=['metadatas'])
            if sample['metadatas']:
                print(f"   📋 ตัวอย่าง metadata: {list(sample['metadatas'][0].keys())}")

    except Exception as e:
        print(f"❌ {collection.name}: Error - {e}")

print(f"\\n=== สรุป ===")
print(f"✅ คอลเลกชันหลัก: 'all_collections_thai'")
print(f"🔤 Embedding Model: sentence-transformers/distiluse-base-multilingual-cased")
print(f"🌐 รองรับภาษาไทยได้ดี")
print(f"📊 รวมข้อมูลจาก 3 แหล่ง: attractions, hotels, restaurants")